# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
import syft as sy
import torch as th

W0731 21:28:04.056895 4584859072 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/miniconda3/envs/pri/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0731 21:28:04.066104 4584859072 deprecation_wrapper.py:119] From /miniconda3/envs/pri/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
hook = sy.TorchHook(th)

In [3]:
from torch import nn, optim

In [4]:
# Create a couple workers
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [5]:
# Make workers know each other
# bob.add_workers([alice, secure_worker])
# alice.add_workers([bob, secure_worker])
# secure_worker.add_workers([bob, alice])

In [6]:
# Same toy dataset for test
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]],requires_grad=True)

In [7]:
# get pointers to training data on each worker by 
# sending some training data to bob and alice

bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [8]:
# Initialize a toy model
model = nn.Linear(2,1)

In [9]:
model.weight.data.set_

<function Tensor.set_>

In [10]:
bob_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

In [11]:
# optimizer sent to bob and alice
bob_optim = optim.SGD(params=bob_model.parameters(), lr=0.1)
alice_optim = optim.SGD(params=alice_model.parameters(), lr=0.1)

In [12]:
# Training

for i in range(20):
    bob_optim.zero_grad()
    bob_pred = bob_model(bob_data)
    bob_loss = ((bob_pred - bob_target)**2).sum()
    bob_loss.backward()
    bob_optim.step()
    bob_loss = bob_loss.get().data

#     print(bob_loss)

    alice_optim.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_loss = ((alice_pred - alice_target)**2).sum()
    alice_loss.backward()
    alice_optim.step()
    alice_loss = alice_loss.get().data

#     print(alice_loss)


In [13]:
bob_model.move(secure_worker)
alice_model.move(secure_worker)

In [14]:
secure_worker

<VirtualWorker id:secure_worker #objects:4>

In [15]:
secure_worker._objects

{39272840950: Parameter containing:
 tensor([[0.2171, 0.1199]], requires_grad=True),
 61640946839: Parameter containing:
 tensor([-0.0741], requires_grad=True),
 98095026948: Parameter containing:
 tensor([[0.8482, 0.1169]], requires_grad=True),
 2711350813: Parameter containing:
 tensor([0.0862], requires_grad=True)}

In [16]:
(bob_model.weight.data + alice_model.weight.data)/2

(Wrapper)>[PointerTensor | me:92253827919 -> secure_worker:70792918132]

In [17]:
bob_model.weight.data

(Wrapper)>[PointerTensor | me:39272840950 -> secure_worker:39272840950]::data

In [18]:
alice_model.weight.data

(Wrapper)>[PointerTensor | me:98095026948 -> secure_worker:98095026948]::data

In [25]:
model.weight.data = ((bob_model.weight.data + alice_model.weight.data)/2).get()

In [20]:
bob_model.bias.data

(Wrapper)>[PointerTensor | me:61640946839 -> secure_worker:61640946839]::data

In [21]:
alice_model.bias.data

(Wrapper)>[PointerTensor | me:2711350813 -> secure_worker:2711350813]::data

In [24]:
model.bias.data = ((bob_model.bias.data + alice_model.bias.data)/2).get()

In [26]:
model.weight.data

tensor([[0.5326, 0.1184]])

In [27]:
model.bias.data

tensor([0.0060])

In [28]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [29]:
alice._objects

{67085346822: tensor([[1., 0.],
         [1., 1.]], requires_grad=True), 77669459338: tensor([[1.],
         [1.]], requires_grad=True), 8633179673: tensor([[0.9280],
         [1.0562]], grad_fn=<AddmmBackward>)}

In [30]:
bob._objects

{93905966369: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 12881787957: tensor([[0.],
         [0.]], requires_grad=True), 87274114242: tensor([[-0.0803],
         [ 0.0496]], grad_fn=<AddmmBackward>)}

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [ ]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [ ]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [ ]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

This even works for addition between two shared values!!

In [ ]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [ ]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

In [ ]:
(bob_x_share + alice_x_share) % Q

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [ ]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [ ]:
# try this project here!

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [ ]:
BASE=10
PRECISION=4

In [ ]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [ ]:
encode(3.5)

In [ ]:
decode(35000)

In [ ]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [ ]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [ ]:
x = x.share(bob, alice, secure_worker)

In [ ]:
bob._objects

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
y.get()

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [ ]:
x = th.tensor([0.1,0.2,0.3])

In [ ]:
x

In [ ]:
x = x.fix_prec()

In [ ]:
x.child.child

In [ ]:
y = x + x

In [ ]:
y = y.float_prec()
y

### Shared Fixed Precision

And of course, we can combine the two!

In [ ]:
x = th.tensor([0.1, 0.2, 0.3])

In [ ]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [ ]:
y = x + x

In [ ]:
y.get().float_prec()

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation